In [1]:
output='''· 제품명제크 · 식품의 유형: 과자(유처리제품) · 롯데제과주식회사 경남 양산시 양산대로 1158 · 직사광선 및
· 영양성분 1회 제공량 1봉 (50g) 습기를 피해 서늘한 곳에 진열하시고, 유통중 변질품은 구입상점 및 본사에서 항상 교환해 드립니다. · 본 제품은
· 총 2회 제공량 (100g) 소비자기본법에 의한 소비자분쟁해결기준에 의거 정당한 소비자 피해에 대하여 교환·환불해 드립니다. · 휴지줍는 고운마을.
· 안버리는 밝은마음 · 유통기한한측면표기일까지(연.월.일) · 내포장제작폴리프로필렌 · 원재료명 밀가루(밀,미국산), 열량
· 탄수화물 33g 10% 가공유지(팜분별유(부분경화유):말레이시아산), 쇼트닝 I (정제가공유지(부분경화유), 당류 4g
· 말레이시아산), 쇼트닝 I (백설탕,산도조절제,액상과당,천일염,쇼트닝 I,맥아분말,치조분말, 단백질 3g 5%
· 레시틴,유청단백분말,효소제,합성착향료(브레드향), 밀,우유,대두,쇠고기,함유
· 포화지방 6g 40% 과자 표면에 있을 수 있는 짧은 실 모양은 맥아껍질이므로 안심하고 종이
· 트랜스지방 0.5g 0% 드시기 바랍니다 · 부정,불량식품 신고는 국번없이 1399
· 콜레스테롤 0mg 0% 콜레스테롤 400mg 20%
· 나트륨 080-024-6060 0% 영양성분기준치:1일 영양성분기준치에 대한 비율
· 수신자요율부담 8 80 1062 52 1906 1400
· 희망소비자가격 http://www.lottecont.co.kr
· 080-024-6060 0% 영양성분기준치:1일 영양성분기준치에 대한 비율<|im_end|>'''

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import json, re

ALLERGENS = [
    "알류","우유","메밀","땅콩","대두","밀","잣","호두",
    "게","새우","오징어","고등어","조개류","복숭아","토마토",
    "닭고기","돼지고기","쇠고기","아황산류"
]



In [4]:
# 1. 모델 로드
try:
    model_id = "paust/pko-t5-small"
    tok = AutoTokenizer.from_pretrained(model_id)
    mdl = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
except:
    model_id = "google/flan-t5-small"
    tok = AutoTokenizer.from_pretrained(model_id)
    mdl = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

gen = pipeline("text2text-generation", model=mdl, tokenizer=tok, max_new_tokens=256)



Device set to use cuda:0


In [5]:
# 2. 추출 함수
def extract_allergens(ocr_text: str):
    prompt = f"""
다음은 OCR로 추출한 식품 정보 라벨 텍스트입니다:
\"{ocr_text}\"

알레르기 카테고리 목록: {', '.join(ALLERGENS)}

👉 포함된 알레르기 카테고리만 JSON 배열로 출력하세요.  
형식: ["밀","대두"]  
⚠️ 정답 JSON만 출력하세요. 다른 설명/문장은 쓰지 마세요.

정답 JSON:"""

    out = gen(prompt)[0]["generated_text"].strip()
    print("📜 Raw Output:", out)  # LLM 실제 응답 확인용

    m = re.search(r"\[.*\]", out, re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except:
            pass

    # fallback: 카테고리 직접 탐색
    found = []
    for a in ALLERGENS:
        if a in ocr_text:
            found.append(a)
    return found


In [6]:
print("✅ Detected Allergens:", extract_allergens(output))

📜 Raw Output: ·  · 당류 0g 10% · 당류 0g 10% · 당류 0g 10% · 당류 0g 10% · 당류 0g 10% · 당류 0g 10% · 당류 0g 10% · 당류 0g 10% · 나트륨 0g 또는또는:
✅ Detected Allergens: ['우유', '대두', '밀', '쇠고기']
